# <center>Organizaci&oacute;n de Datos</center>
#### <center>C&aacute;tedra Ing. Rodriguez, Juan Manuel </center>

## <center>Feature Engineering</center>
### <center> Práctica Valores Faltantes</center>

In [ ]:
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import sklearn.preprocessing as skp 

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

pd.options.display.max_columns = None

#### Preparación del Dataset

In [ ]:
df=pd.read_csv("./ar_properties.csv")

#### Verificamos la calidad de los datos

Cómo vimos en la teórica los valores nulos pueden tener distintas formas de ser represantados:

* nan
* vacios ""
* algun caracter especial "_", "?", "NULL"
* valores que no tienen sentido dado la variable (ej: distancia recorrida: -1)

Vamos a analizar el dataset general

In [ ]:
tienen_caracter_raro = df.astype('str').eq('-').any(0)
tienen_caracter_raro

id                 False
ad_type            False
start_date         False
end_date           False
created_on         False
lat                False
lon                False
l1                 False
l2                 False
l3                 False
l4                 False
l5                 False
l6                 False
rooms              False
bedrooms           False
bathrooms          False
surface_total      False
surface_covered    False
price              False
currency           False
price_period       False
title              False
property_type      False
operation_type     False
dtype: bool

Chequemos variables numéricas

In [ ]:
columnas_con_numeros = ['price']
(df[columnas_con_numeros] <= 0).any().to_frame('Menor que 0 ?')

,Menor que 0 ?
price,True


In [ ]:
filas_totales=df.shape[0]
print(df.isna().sum()/filas_totales*100)

id                   0.000000
ad_type              0.000000
start_date           0.000000
end_date             0.000000
created_on           0.000000
lat                 13.010587
lon                 13.010587
l1                   0.000000
l2                   0.000000
l3                   3.283954
l4                  70.297333
l5                  99.381318
l6                 100.000000
rooms               37.200141
bedrooms            59.334621
bathrooms           24.206269
surface_total       19.044668
surface_covered     25.162321
price                5.457056
currency             7.455560
price_period        41.448375
title                0.000000
property_type        0.000000
operation_type       0.000000
dtype: float64


#### Duplicados:
Dependiendo del dataset, a veces tenemos informacion duplicada que no queremos
Alertamos una fila duplicada, la eliminamos

df.duplicated() -> devuelve una serie de booleanos indicando se una fila es duplicada o no

df.drop_duplicates() -> devuelve un dataframe nuevo con las filas duplicadas eliminadas

Nota: podemos pasarle el parametro subset=\ para solo considerar algunas columnas para ver si esta duplicado o no

In [ ]:
df[df.duplicated(keep=False)]

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,l2,l3,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,property_type,operation_type


In [ ]:
size_antes = len(df_filtrado)
df_filtrado = df_filtrado.drop_duplicates()
size_despues = len(df_filtrado)
print(f'Se eliminaron: {size_antes-size_despues} filas duplicadas')

Se eliminaron: 0 filas duplicadas


Nota: a veces es util "resetear" el indice despues de eliminar filas (ya sea por drop duplicates o por algun otro filtro)

In [ ]:
df_filtrado.reset_index(drop=True, inplace=True)

Una vez analizados los datos, tenemos que tomar desiciones sobre qué hacer con los datos faltantes

#### Opción 0: Eliminarlos del dataset

Esta opción la vimos algunas clases atrás
( Vamos a trabajar con una versión reducida del dataset, seleccionando sólo algunas columnas )

In [ ]:
df_filtrado=df.copy()

cond_lugar=df_filtrado['l2']== "Capital Federal"
cond_moneda=df_filtrado['currency']== "USD"
cond_operacion=df_filtrado['operation_type']=="Venta"

df_filtrado=df_filtrado[cond_lugar & cond_moneda & cond_operacion ]

columnas={"l2":"ciudad", "l3":"barrio"}
df_filtrado.rename(columns = columnas, inplace = True)

In [ ]:
df_filtrado.head()

,id,ad_type,start_date,end_date,created_on,lat,lon,l1,ciudad,barrio,l4,l5,l6,rooms,bedrooms,bathrooms,surface_total,surface_covered,price,currency,price_period,title,property_type,operation_type
242,lnQPuBQdzGwJ9akSrlt18Q==,Propiedad,2019-04-14,2019-06-23,2019-04-14,-34.597732,-58.380247,Argentina,Capital Federal,Retiro,NaN,NaN,NaN,NaN,NaN,NaN,13.0,NaN,22000.0,USD,NaN,VENTA - COCHERA - RETIRO,Cochera,Venta
243,4XQ+AhiGd1fjqaMBs7wKJQ==,Propiedad,2019-04-14,2019-04-17,2019-04-14,-34.599240,-58.426516,Argentina,Capital Federal,Almagro,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22000.0,USD,Mensual,COCHERA EN VENTA,Cochera,Venta
247,sPaaKKHnzVV2hllqm7LHIA==,Propiedad,2019-04-14,2019-06-23,2019-04-14,-34.592054,-58.388566,Argentina,Capital Federal,Recoleta,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,45500.0,USD,NaN,Cochera DOBLE de 26 m2 totales en RECOLETA,Cochera,Venta
248,2Onpsu8EJMq1OIYuguv9+Q==,Propiedad,2019-04-14,2019-08-17,2019-04-14,-34.599638,-58.390532,Argentina,Capital Federal,Barrio Norte,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.0,23500.0,USD,NaN,COCHERA EN AV. CORDOBA 1500,Cochera,Venta
249,BZR4Z16so70uY4h7R8dbpg==,Propiedad,2019-04-14,2019-06-23,2019-04-14,-34.592512,-58.389092,Argentina,Capital Federal,Barrio Norte,NaN,NaN,NaN,NaN,NaN,NaN,26.0,NaN,45500.0,USD,NaN,Cochera DOBLE de 26 m2 totales en Barrio Norte,Cochera,Venta


In [ ]:
df_eliminar_nans=df_filtrado.copy()

In [ ]:
filas_totales=df_eliminar_nans.shape[0]
print(df_eliminar_nans.isna().sum()/filas_totales*100)

id                   0.000000
ad_type              0.000000
start_date           0.000000
end_date             0.000000
created_on           0.000000
lat                  4.415783
lon                  4.415783
l1                   0.000000
ciudad               0.000000
barrio               0.589307
l4                  96.699882
l5                 100.000000
l6                 100.000000
rooms               22.042752
bedrooms            50.492875
bathrooms           15.056788
surface_total        7.095789
surface_covered      9.890978
price                0.000000
currency             0.000000
price_period        41.014411
title                0.000000
property_type        0.000000
operation_type       0.000000
dtype: float64


Podemos pensar en eliminar las siguientes columnas

In [ ]:
columnas_eliminar_NANs=['bedrooms','l4','l5','l6']
df_eliminar_nans.drop(columnas_eliminar_NANs, axis='columns', inplace=True)

In [ ]:
filas_totales=df_eliminar_nans.shape[0]
print(df_eliminar_nans.isna().sum()/filas_totales*100)

id                  0.000000
ad_type             0.000000
start_date          0.000000
end_date            0.000000
created_on          0.000000
lat                 4.415783
lon                 4.415783
l1                  0.000000
ciudad              0.000000
barrio              0.589307
rooms              22.042752
bathrooms          15.056788
surface_total       7.095789
surface_covered     9.890978
price               0.000000
currency            0.000000
price_period       41.014411
title               0.000000
property_type       0.000000
operation_type      0.000000
dtype: float64


Eliminamos el resto de las filas con Nans

In [ ]:
df_eliminar_nans.dropna(inplace=True)

In [ ]:
print("cantidad de registros originales: "+str(df_filtrado.shape[0]))
print("cantidad de registros finals: "+str(df_eliminar_nans.shape[0]))

porcentaje=(df_filtrado.shape[0]-df_eliminar_nans.shape[0])/df_filtrado.shape[0]*100

print("Eliminamos el "+str(porcentaje)+" % de los registros")

cantidad de registros originales: 74664
cantidad de registros finals: 28574
Eliminamos el 61.72988321011464 % de los registros


#### Opción 1: Tratarla como una "categoria" o valor más

In [ ]:
df_eliminar_nans_op1=df_filtrado.copy()

df_eliminar_nans_op1['bathrooms'] = df_eliminar_nans_op1['bathrooms'].fillna(99)

df_eliminar_nans_op1['barrio'] = df_eliminar_nans_op1['barrio'].fillna("Barrio no existe")

#### Opcion 2: completar usando info de esa columna (Univariadas)

In [ ]:
df_eliminar_nans_op2=df_filtrado.copy()

Completar con la mediana, promedio, moda o constante

In [ ]:
# Devuelve el valor de imputacion de las tres estrategias para esa columna

def show_strategies(df, name_col, k=99):
    _df = df[[name_col]].copy()
    s = df[name_col]

    _df['mediana'] = s.fillna(s.median())
    _df['media'] = s.fillna(s.mean())
    _df['moda'] = s.fillna(s.mode()[0])
    _df['constante'] = k

    return _df[s.isna()]

show_strategies(df_eliminar_nans_op2, 'rooms',99)

,rooms,mediana,media,moda,constante
0,NaN,3.0,2.806652,3.0,99
1,NaN,3.0,2.806652,3.0,99
2,NaN,3.0,2.806652,3.0,99
3,NaN,3.0,2.806652,3.0,99
4,NaN,3.0,2.806652,3.0,99
...,...,...,...,...,...
388459,NaN,3.0,2.806652,3.0,99
388479,NaN,3.0,2.806652,3.0,99
388708,NaN,3.0,2.806652,3.0,99
388848,NaN,3.0,2.806652,3.0,99


##### ToDo (Tarea)
Reemplazar los datos faltantes en variable rooms por cada uno de estas estratégicas y graficar la distribución de la variable antes y despúes de realizar la imputación

In [1]:
_df.head()

NameError: ignored

#### Opcion 3: completar usando info de las demas columnas (Multivariada)

In [ ]:
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

columnas=['surface_total','surface_covered','bathrooms','rooms']
df_eliminar_nans_op3=df_filtrado[columnas].copy()

lr = LinearRegression()
imp = IterativeImputer(estimator=lr,missing_values=np.nan, max_iter=20, verbose=0, random_state=0) #imputation_order='roman'
df_eliminar_nans_op3_completo=imp.fit_transform(df_eliminar_nans_op3)

In [ ]:
df_eliminar_nans_op3_completo

array([[ 13.        ,  79.38717066,   1.        ,   2.30213788],
       [111.85778578, 119.73515557,   1.        ,   2.30492748],
       [ 26.        ,  84.69303181,   1.        ,   2.30250472],
       ...,
       [284.        , 182.        ,   3.        ,   6.        ],
       [601.        , 400.        ,   5.        ,   8.        ],
       [187.        , 152.        ,   3.        ,   9.        ]])